## Type validation decorator
#### Provides one-liner to validate the types of the kwargs passed to a function

In [65]:
from functools import wraps

def validate_types(**types):
    def decorator(foo):
        @wraps(foo)
        def validator(*args, **kwargs):
            errors = []
            for param, value in kwargs.items():
                expected = types.get(param, object)
                if not isinstance(expected, list):
                    expected = [expected, ]
                if not any([isinstance(value, t) for t in expected]):
                    actual = type(value)
                    errors.append({
                        'param_name': param,
                        'param_value': value,
                        'reason': 'Expected {}, got {}'.format(expected, actual)
                    })
            if errors:
                raise ValueError(errors)
            return foo(*args, **kwargs)
        return validator
    return decorator

## Usage
<ol>
    <li>Can pass either a single type or a list of allowed types</li>
    <li>Works with kwargs only, both required and optional</li>
    <li>Validates only explicitly passed kwargs</li>
    <li>Ignoring any other positional arguments and kwargs</li>
</ol>

In [66]:
@validate_types(a=[int, float], b=int, kwarg=[str, type(None)])
def foo(arg, kwarg=None, *, a, b):  # <a>, <b> -> required kwargs; <kwarg> -> optional kwarg
    return a + b


@validate_types(a=str, b=str)
def bar(*, a, b):
    return a + b

## Functions call / errors examples:

In [67]:
foo(42, kwarg='some_param', a=42.2, b=42), foo(42, a=42, b=42)

(84.2, 84)

In [68]:
foo(a='42.2', b=42.42)

ValueError: [{'param_value': '42.2', 'param_name': 'a', 'reason': "Expected [<class 'int'>, <class 'float'>], got <class 'str'>"}, {'param_value': 42.42, 'param_name': 'b', 'reason': "Expected [<class 'int'>], got <class 'float'>"}]

In [69]:
bar(a='Hello ', b='World!')

'Hello World!'

In [70]:
bar(a='42', b=1.4)

ValueError: [{'param_value': 1.4, 'param_name': 'b', 'reason': "Expected [<class 'str'>], got <class 'float'>"}]

## Perfomance 

In [77]:
from random import randint as ri

@validate_types(a=[int, float], b=int, kwarg=[str, type(None)])
def decorated_perfomance_test(arg, kwarg=None, *, a, b):  
    return [ri(1,10) for i in range(10**3)]


def perfomance_test(arg, kwarg=None, *, a, b):  
    return [ri(1,10) for i in range(10**3)]

% timeit decorated_perfomance_test(42, kwarg='some_param', a=42.2, b=42)

1000 loops, best of 3: 1.23 ms per loop


In [78]:
% timeit perfomance_test(42, kwarg='some_param', a=42.2, b=42)

1000 loops, best of 3: 1.21 ms per loop


In [79]:
% timeit [ri(1,10) for i in range(10**3)]

1000 loops, best of 3: 1.21 ms per loop
